In [2]:
#loading modules and  packages
import logging
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
from sklearn.linear_model import Ridge
from azureml.core.model import Model

from azureml.core import Datastore

# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

default_ds.upload_files(files=['./data/diabetes.csv'], # Upload the data/diabetes.csv file
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)
                       
from azureml.core import Dataset

#Create a tabular dataset from the path on the datastore (this may take a short while)
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/diabetes.csv'))

# Display the first 20 rows as a Pandas dataframe
data_set.take(20).to_pandas_dataframe()


# Register the dataset
dataset_name = 'Diabetes Dataset'
data_set = data_set.register(workspace=ws, 
                           name=dataset_name,
                           description='diabetes data',
                           tags = {'year':'2019', 'category':'Diabetes'},
                           create_new_version=True)

# List the datasets registered in the workspace
for ds in ws.datasets:
    print(ds)

In [3]:
ws = Workspace.from_config()

In [4]:
print(ws)

Workspace.create(name='xm-ml-workspace', subscription_id='be564fde-136b-4709-b7b6-abfc0bdfc134', resource_group='xm-ml')


# Best Model Based on Any Other Metric
#Show the run and the model which has the smallest log_loss va
lookup_metric = "log_loss"
best_run, fitted_model = remote_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

# Model from a specfic iteration
iteration = 1
third_run, third_model = remote_run.get_output(iteration=iteration)
print(third_run)
print(third_model)

class_models = Model.list(workspace=ws, tags=['area'])
for m in class_models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

In [ ]:
#registering the model
model = Model.register(model_name = "riskprediction",
                       model_path = "model.pkl",
                       tags = {"key": "1","fraud": "riskpredict"},
                       description = "riskpredictionanalysis",
                       workspace = ws)

In [5]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame(data=[{"TOTAL_PRICE":869.92,"THIRD_PARTY_ID_SCORE":415,"IS_EXISTING_CUSTOMER":"Y","NUM_PORTIN":0,"FIRST_PARTY_ID_SCORE":356,"ONETIMECHARGE":33.33,"INSTALLMENT_AMOUNT":33.33,"DEVICE_AT_HOME":"Y","FRAUDNET_SCORE":200.0,"NUM_BYOD":0,"IDA_RESULT":"GREEN","EXTERNAL_CREDIT_CHECK_DONE":"Y","SALES_CHANNEL":"ONLINE","MODEL1":"iPhone XR","MODEL2":"NA","MODEL3":"NA","MODEL4":"NA","MODEL5":"NA"}])
output_sample = np.array([0])


def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'riskprediction')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        if result ==0:
            result = 'GREEN'
            print('GREEN')
        elif result ==1:
            result = 'YELLOW'
            print('YELLOW')
        else:
            result = 'RED'
            print('RED')
        print('result') 
        fraud_status =[]
        fraud_status.append(result)
        return fraud_status
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result": result.tolist()})


Overwriting score.py


In [6]:
#create environment file
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
fraudenv = CondaDependencies()
fraudenv.add_conda_package("scikit-learn")
fraudenv.add_conda_package("numpy")
fraudenv.add_conda_package("scipy")
fraudenv.add_conda_package("pandas")
fraudenv.add_conda_package("py-xgboost<=0.80")
fraudenv.add_pip_package("azureml-defaults")
fraudenv.add_pip_package("azureml-train-automl==1.0.55")
fraudenv.add_pip_package("azureml-core==1.0.55")
fraudenv.add_pip_package("inference-schema")
fraudenv.conda_dependencies_file_path="data/env_dependencies.json"
with open("riskpredictionenv.yml","w") as f:
    f.write(fraudenv.serialize_to_string())

In [1]:
#create AKS cluster( its one time creation; No need to create for each deployment)
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
prov_config = AksCompute.provisioning_configuration()
prov_config.enable_ssl(leaf_domain_label = "riskprediction")

aks_name = 'riskaks'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

NameError: name 'ws' is not defined

In [ ]:
(aks_name)

In [ ]:
# creating a container image
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="riskpredictionenv.yml",
                                 tags = {"key": "1"},
                                 description = "riskpredictionanalysis")
image = Image.create(name = "riskprediction",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

In [ ]:
print(image_config)

In [ ]:
#create inference config
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="riskpredictionenv.yml")

In [ ]:
model

In [ ]:
#service deployment
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.image import Image, ContainerImage

aks_target = AksCompute(ws,"riskpredictdemo1")

# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights=True)
service = Model.deploy(ws, "riskpredictionanalysisnew", [model] , inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

In [ ]:
print(deployment_config)



In [ ]:
print(service.get_logs())

In [ ]:
print("The score uri:",service.scoring_uri)
print("The swagger:", service.swagger_uri)
primary, secondary = service.get_keys()
print("the primary key:",primary)
print("the secondry key:",secondary)

In [ ]:
print(service.swagger_uri)

In [ ]:
#things to do
how to retireve the existing AKS information

In [ ]:
primary, secondary = service.get_keys()
print(primary)

In [ ]:
print(service.get_logs())

In [ ]:
print(secondary)